In [2]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import sklearn
import sklearn.datasets
from numpy.random import seed
seed(1)
import pandas as pd


In [3]:
!pydotplus

/bin/sh: pydotplus: command not found


In [4]:
data_df = pd.read_csv("../db/stock_data_with_perf_fixed.csv")
data_df.columns

/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2698: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


Index(['name', 'ticker', 'Quarter end', 'Shares', 'Shares split adjusted',
       'Split factor', 'Assets', 'Current Assets', 'Liabilities',
       'Current Liabilities', 'Shareholders equity',
       'Non-controlling interest', 'Preferred equity',
       'Goodwill & intangibles', 'Long-term debt', 'Revenue', 'Earnings',
       'Earnings available for common stockholders', 'EPS basic',
       'EPS diluted', 'Dividend per share', 'Cash from operating activities',
       'Cash from investing activities', 'Cash from financing activities',
       'Cash change during period', 'Cash at end of period',
       'Capital expenditures', 'Price', 'Price high', 'Price low', 'ROE',
       'ROA', 'Book value of equity per share', 'P/B ratio', 'P/E ratio',
       'Cumulative dividends per share', 'Dividend payout ratio',
       'Long-term debt to equity ratio', 'Equity to assets ratio',
       'Net margin', 'Asset turnover', 'Free cash flow per share',
       'Current ratio', 'mktcap_revenue_value', '

In [21]:
data_df.head()

,name,ticker,Quarter end,Shares,Shares split adjusted,Split factor,Assets,Current Assets,Liabilities,Current Liabilities,...,one_year_return,two_year_return,three_year_return,four_year_return,five_year_return,seven_year_return,ten_year_return,twelve_year_return,fifteen_year_return,Value_stock
0,NaN,CRA,2011-03-26,82181869,82181869,1,607664000,364389000,44109000,29630000,...,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,NaN,CRA,2010-12-25,82113206,82113206,1,612278000,365667000,44805000,30514000,...,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,NaN,CRA,2010-09-25,82078098,82078098,1,617005000,359707000,41036000,26951000,...,9.138187,-47.185081,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,NaN,CRA,2010-06-26,81987689,81987689,1,625593000,366249000,44013000,30068000,...,-0.512164,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,NaN,CRA,2010-03-27,81987689,81987689,1,625022000,363068000,39415000,25625000,...,-8.900836,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [5]:
#Get the features data to be Used
#specify the features(measures) and target (label) to be used
features_columns = ['EPS basic','ROE','ROA', 'P/B ratio', 'P/E ratio', 'Dividend payout ratio',
                   'Long-term debt to equity ratio', 'Equity to assets ratio', 'Free cash flow per share',
                   'Current ratio', 'mktcap_revenue_value', 'mktcap_free_cash_flow_value',
                   'mktcap_cash_value', 'cash_oper_gt_earnings_value',
                   'entvalue_earnings_value', 'marketcap_bookvalue_value']

features_data_df = data_df[features_columns]
data = features_data_df.values

#get target data
target_values = ['no','yes']

selected_target_column = 'five_year_return'

data_df.loc[data_df[selected_target_column] > 0.0, 'Value_stock'] = 1
data_df.loc[data_df[selected_target_column] <= 0.0, 'Value_stock'] = 0
target = data_df['Value_stock'].values

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(data, target, random_state=42)

In [6]:
 from sklearn.preprocessing import StandardScaler

# Create a StandardScater model and fit it to the training data
X_scaler = StandardScaler().fit(X_train)
X_scaler

StandardScaler(copy=True, with_mean=True, with_std=True)

In [7]:
#Transform the training and the testing data using the X_scaler
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [8]:
!pip install keras

In [9]:
from keras.utils import to_categorical

# One-hot encoding
y_train_categorical = to_categorical(y_train)
y_test_categorical = to_categorical(y_test)

/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [10]:
# first, create a normal neural network with 2 inputs, 6 hidden nodes, and 2 outputs
from keras.models import Sequential
from keras.layers import Dense

model = Sequential()
model.add(Dense(units=6, activation='relu', input_dim=16))
model.add(Dense(units=2, activation='softmax'))

In [11]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 6)                 102       
_________________________________________________________________
dense_2 (Dense)              (None, 2)                 14        
Total params: 116
Trainable params: 116
Non-trainable params: 0
_________________________________________________________________


In [12]:
# Compile the model
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [13]:
 # Fit the model to the training data
model.fit(
    X_train_scaled,
    y_train_categorical,
    epochs=100,
    shuffle=True,
    verbose=2
)


Epoch 1/100
 - 2s - loss: 0.6646 - acc: 0.6243
Epoch 2/100
 - 2s - loss: 0.6354 - acc: 0.6707
Epoch 3/100
 - 2s - loss: 0.6316 - acc: 0.6763
Epoch 4/100
 - 2s - loss: 0.6296 - acc: 0.6775
Epoch 5/100
 - 2s - loss: 0.6272 - acc: 0.6767
Epoch 6/100
 - 2s - loss: 0.6244 - acc: 0.6769
Epoch 7/100
 - 2s - loss: 0.6227 - acc: 0.6771
Epoch 8/100
 - 2s - loss: 0.6210 - acc: 0.6776
Epoch 9/100
 - 2s - loss: 0.6201 - acc: 0.6780
Epoch 10/100
 - 2s - loss: 0.6194 - acc: 0.6766
Epoch 11/100
 - 2s - loss: 0.6186 - acc: 0.6789
Epoch 12/100
 - 2s - loss: 0.6184 - acc: 0.6772
Epoch 13/100
 - 2s - loss: 0.6180 - acc: 0.6774
Epoch 14/100
 - 2s - loss: 0.6175 - acc: 0.6777
Epoch 15/100
 - 2s - loss: 0.6173 - acc: 0.6782
Epoch 16/100
 - 2s - loss: 0.6170 - acc: 0.6785
Epoch 17/100
 - 2s - loss: 0.6166 - acc: 0.6790
Epoch 18/100
 - 2s - loss: 0.6164 - acc: 0.6777
Epoch 19/100
 - 2s - loss: 0.6161 - acc: 0.6779
Epoch 20/100
 - 2s - loss: 0.6160 - acc: 0.6789
Epoch 21/100
 - 2s - loss: 0.6161 - acc: 0.6785
E

In [14]:
deep_model = Sequential()
deep_model.add(Dense(units=6, activation='relu', input_dim=16))
deep_model.add(Dense(units=6, activation='relu'))
deep_model.add(Dense(units=2, activation='softmax'))

In [15]:
deep_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_3 (Dense)              (None, 6)                 102       
_________________________________________________________________
dense_4 (Dense)              (None, 6)                 42        
_________________________________________________________________
dense_5 (Dense)              (None, 2)                 14        
Total params: 158
Trainable params: 158
Non-trainable params: 0
_________________________________________________________________


In [16]:
deep_model.compile(optimizer='adam',
                   loss='categorical_crossentropy',
                   metrics=['accuracy'])

deep_model.fit(
    X_train_scaled,
    y_train_categorical,
    epochs=100,
    shuffle=True,
    verbose=2
)

Epoch 1/100
 - 3s - loss: 0.6477 - acc: 0.6356
Epoch 2/100
 - 3s - loss: 0.6240 - acc: 0.6739
Epoch 3/100
 - 3s - loss: 0.6188 - acc: 0.6785
Epoch 4/100
 - 3s - loss: 0.6149 - acc: 0.6828
Epoch 5/100
 - 3s - loss: 0.6118 - acc: 0.6867
Epoch 6/100
 - 3s - loss: 0.6099 - acc: 0.6884
Epoch 7/100
 - 3s - loss: 0.6086 - acc: 0.6896
Epoch 8/100
 - 3s - loss: 0.6076 - acc: 0.6901
Epoch 9/100
 - 3s - loss: 0.6065 - acc: 0.6905
Epoch 10/100
 - 3s - loss: 0.6063 - acc: 0.6925
Epoch 11/100
 - 3s - loss: 0.6055 - acc: 0.6921
Epoch 12/100
 - 3s - loss: 0.6047 - acc: 0.6924
Epoch 13/100
 - 3s - loss: 0.6043 - acc: 0.6924
Epoch 14/100
 - 3s - loss: 0.6038 - acc: 0.6927
Epoch 15/100
 - 3s - loss: 0.6030 - acc: 0.6919
Epoch 16/100
 - 3s - loss: 0.6024 - acc: 0.6927
Epoch 17/100
 - 2s - loss: 0.6020 - acc: 0.6922
Epoch 18/100
 - 3s - loss: 0.6015 - acc: 0.6921
Epoch 19/100
 - 3s - loss: 0.6011 - acc: 0.6924
Epoch 20/100
 - 2s - loss: 0.6003 - acc: 0.6927
Epoch 21/100
 - 2s - loss: 0.5996 - acc: 0.6930
E

In [17]:
model_loss, model_accuracy = model.evaluate(
    X_test_scaled, y_test_categorical, verbose=2)
print(
    f"Normal Neural Network - Loss: {model_loss}, Accuracy: {model_accuracy}") 

Normal Neural Network - Loss: 0.6100854520543937, Accuracy: 0.6880928011121135


In [18]:
model_loss, model_accuracy = deep_model.evaluate(
    X_test_scaled, y_test_categorical, verbose=2)
print(f"Deep Neural Network - Loss: {model_loss}, Accuracy: {model_accuracy}")

Deep Neural Network - Loss: 0.588212443471809, Accuracy: 0.6981235072987757


In [23]:
#use Rajesh's best picked colums to check for accuracy
#or pick them at random 8 12 14 and 5 to see how it works
#pick some of them at random

reduced_data_df = pd.read_csv("../db/stock_data_reducedcol (2).csv")
reduced_data_df.head()

,Unnamed: 0,name,ticker,Quarter end,EPS basic,Price,Price high,Price low,ROE,ROA,P/B ratio,P/E ratio,one_year_return,two_year_return,three_year_return,four_year_return,five_year_return
0,0,NaN,CRA,2011-03-26,-0.06,7.17,8.42,5.92,-0.0305,-0.0283,1.04,0.0,0.000000,0.000000,0.0,0.0,0.0
1,1,NaN,CRA,2010-12-25,0.02,6.17,6.98,5.35,-0.0426,-0.0397,0.88,0.0,0.000000,0.000000,0.0,0.0,0.0
2,2,NaN,CRA,2010-09-25,-0.10,6.69,7.25,6.13,-0.1669,-0.1556,0.94,0.0,9.138187,-47.185081,0.0,0.0,0.0
3,3,NaN,CRA,2010-06-26,-0.07,7.11,7.72,6.49,-0.1390,-0.1294,1.00,0.0,-0.512164,0.000000,0.0,0.0,0.0
4,4,NaN,CRA,2010-03-27,-0.15,6.71,7.49,5.94,-0.0742,-0.0690,0.93,0.0,-8.900836,0.000000,0.0,0.0,0.0


In [26]:
reduced_data_df.columns

Index(['Unnamed: 0', 'name', 'ticker', 'Quarter end', 'EPS basic', 'Price',
       'Price high', 'Price low', 'ROE', 'ROA', 'P/B ratio', 'P/E ratio',
       'one_year_return', 'two_year_return', 'three_year_return',
       'four_year_return', 'five_year_return'],
      dtype='object')

In [27]:
#Get featured columns to be used for this 
featured_reduced_columns = ['Unnamed: 0', 'name', 'ticker', 'Quarter end', 'EPS basic', 'Price',
       'Price high', 'Price low', 'ROE', 'ROA', 'P/B ratio', 'P/E ratio',
       'one_year_return', 'two_year_return', 'three_year_return',
       'four_year_return', 'five_year_return']

features_reduced_data_df = data_df[features_columns]
data = features_reduced_data_df.values

#get target data
target_values = ['no','yes']

selected_target_column = 'five_year_return'

data_df.loc[data_df[selected_target_column] > 0.0, 'Value_stock'] = 1
data_df.loc[data_df[selected_target_column] <= 0.0, 'Value_stock'] = 0
target = data_df['Value_stock'].values

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(data, target, random_state=42)


In [28]:
from sklearn.preprocessing import StandardScaler

# Create a StandardScater model and fit it to the training data
X_scaler = StandardScaler().fit(X_train)
X_scaler

StandardScaler(copy=True, with_mean=True, with_std=True)

In [29]:
#Transform the training and the testing data using the X_scaler
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [30]:
from keras.utils import to_categorical

# One-hot encoding
y_train_categorical = to_categorical(y_train)
y_test_categorical = to_categorical(y_test)

In [50]:
# first, create a normal neural network with 2 inputs, 6 hidden nodes, and 2 outputs
from keras.models import Sequential
from keras.layers import Dense

model_two_reduced = Sequential()
model_two_reduced.add(Dense(units=6, activation='relu', input_dim=16))
model_two_reduced.add(Dense(units=2, activation='softmax'))

In [51]:
model_two_reduced.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_20 (Dense)             (None, 6)                 102       
_________________________________________________________________
dense_21 (Dense)             (None, 2)                 14        
Total params: 116
Trainable params: 116
Non-trainable params: 0
_________________________________________________________________


In [52]:
# Compile the model
model_two_reduced.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [53]:
 # Fit the model to the training data
model_two_reduced.fit(
    X_train_scaled,
    y_train_categorical,
    epochs=100,
    shuffle=True,
    verbose=2
)


Epoch 1/100
 - 3s - loss: 0.6662 - acc: 0.6444
Epoch 2/100
 - 2s - loss: 0.6340 - acc: 0.6731
Epoch 3/100
 - 2s - loss: 0.6297 - acc: 0.6801
Epoch 4/100
 - 2s - loss: 0.6271 - acc: 0.6805
Epoch 5/100
 - 2s - loss: 0.6253 - acc: 0.6800
Epoch 6/100
 - 2s - loss: 0.6235 - acc: 0.6801
Epoch 7/100
 - 2s - loss: 0.6220 - acc: 0.6806
Epoch 8/100
 - 3s - loss: 0.6206 - acc: 0.6816
Epoch 9/100
 - 2s - loss: 0.6194 - acc: 0.6822
Epoch 10/100
 - 2s - loss: 0.6182 - acc: 0.6817
Epoch 11/100
 - 2s - loss: 0.6175 - acc: 0.6827
Epoch 12/100
 - 2s - loss: 0.6163 - acc: 0.6824
Epoch 13/100
 - 2s - loss: 0.6155 - acc: 0.6839
Epoch 14/100
 - 2s - loss: 0.6146 - acc: 0.6841
Epoch 15/100
 - 3s - loss: 0.6135 - acc: 0.6843
Epoch 16/100
 - 3s - loss: 0.6125 - acc: 0.6831
Epoch 17/100
 - 2s - loss: 0.6121 - acc: 0.6848
Epoch 18/100
 - 2s - loss: 0.6114 - acc: 0.6851
Epoch 19/100
 - 2s - loss: 0.6104 - acc: 0.6852
Epoch 20/100
 - 2s - loss: 0.6100 - acc: 0.6846
Epoch 21/100
 - 2s - loss: 0.6099 - acc: 0.6851
E

In [54]:
deep_model_reduced = Sequential()
deep_model_reduced.add(Dense(units=6, activation='relu', input_dim=16))
deep_model_reduced.add(Dense(units=6, activation='relu'))
deep_model_reduced.add(Dense(units=2, activation='softmax'))

In [55]:
deep_model_reduced.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_22 (Dense)             (None, 6)                 102       
_________________________________________________________________
dense_23 (Dense)             (None, 6)                 42        
_________________________________________________________________
dense_24 (Dense)             (None, 2)                 14        
Total params: 158
Trainable params: 158
Non-trainable params: 0
_________________________________________________________________


In [62]:
deep_model_reduced.compile(optimizer='adam',
                   loss='categorical_crossentropy',
                   metrics=['accuracy'])

deep_model_reduced.fit(
    X_train_scaled,
    y_train_categorical,
    epochs=100,
    shuffle=True,
    verbose=2
)

Epoch 1/100
 - 3s - loss: 0.6480 - acc: 0.6472
Epoch 2/100
 - 2s - loss: 0.6258 - acc: 0.6738
Epoch 3/100
 - 2s - loss: 0.6198 - acc: 0.6768
Epoch 4/100
 - 2s - loss: 0.6165 - acc: 0.6814
Epoch 5/100
 - 2s - loss: 0.6136 - acc: 0.6866
Epoch 6/100
 - 2s - loss: 0.6111 - acc: 0.6880
Epoch 7/100
 - 2s - loss: 0.6097 - acc: 0.6890
Epoch 8/100
 - 2s - loss: 0.6085 - acc: 0.6896
Epoch 9/100
 - 2s - loss: 0.6076 - acc: 0.6906
Epoch 10/100
 - 2s - loss: 0.6062 - acc: 0.6906
Epoch 11/100
 - 3s - loss: 0.6055 - acc: 0.6916
Epoch 12/100
 - 3s - loss: 0.6052 - acc: 0.6912
Epoch 13/100
 - 3s - loss: 0.6046 - acc: 0.6918
Epoch 14/100
 - 2s - loss: 0.6045 - acc: 0.6924
Epoch 15/100
 - 3s - loss: 0.6043 - acc: 0.6920
Epoch 16/100
 - 3s - loss: 0.6035 - acc: 0.6927
Epoch 17/100
 - 3s - loss: 0.6032 - acc: 0.6932
Epoch 18/100
 - 3s - loss: 0.6023 - acc: 0.6928
Epoch 19/100
 - 3s - loss: 0.6020 - acc: 0.6932
Epoch 20/100
 - 3s - loss: 0.6016 - acc: 0.6939
Epoch 21/100
 - 3s - loss: 0.6012 - acc: 0.6937
E

In [63]:
model_loss, model_accuracy = model_two_reduced.evaluate(
    X_test_scaled, y_test_categorical, verbose=2)
print(
    f"Normal Neural Network - Loss for reduced data: {model_loss}, Accuracy: {model_accuracy}") 

Normal Neural Network - Loss for reduced data: 0.6088978301254566, Accuracy: 0.6930740361895344


In [64]:
model_loss, model_accuracy = deep_model_reduced.evaluate(
    X_test_scaled, y_test_categorical, verbose=2)
print(f"Deep Neural Network - Loss: {model_loss}, Accuracy: {model_accuracy}")

Deep Neural Network - Loss: 0.5900269579822315, Accuracy: 0.6969634936515563
